# Sample Notebook for exploring gnomAD in BigQuery
This notebook contains sample queries to explore the gnomAD dataset which is hosted through the Google Cloud Public Datasets Program.

## Setup and Authentication

If you just want to look at sample results, you can scroll down to see the output of the existing queries without having to run anything. If you would like to re-run the queries or make changes, you will need to authenticate as your user and set the Google Cloud project in which to run the analysis.

In [1]:
# Import libraries
import numpy as np
import os

# Imports for using and authenticating BigQuery
from google.colab import auth

### User Authentication
Before running any queries using BigQuery, you need to first authenticate yourself by running the following cell. If you are running it for the first time, it will ask you to follow a link to log in using your Google identity account, and accept the data access requests to your profile. Once this is done, it will generate a string of verification code, which you should paste back to the cell below and press enter. This should be a Google account which you can login to and which has access to run BigQuery jobs in the Google Cloud project specified in the next step.

In [2]:
auth.authenticate_user()

### Set Google Cloud Project
To run queries in BigQuery, you need to specify the Google Cloud project that will be used. The queries below report the number of bytes billed by each query. The first 1 TB of query data processed in a project per month is free. For more details, see the [BigQuery Pricing](https://cloud.google.com/bigquery/pricing) page.

To find your Project ID, go to the [Project Settings page](https://console.cloud.google.com/iam-admin/settings) in the Google [Cloud Console](https://console.cloud.google.com/). You can select the project you want using the drop-down menu at the top of the page.

In [3]:
# Replace project_id with your Google Cloud Project ID. 
os.environ["GOOGLE_CLOUD_PROJECT"]='project-id'

# gnomAD Queries Type1: Explore a particular genomic region
This category include queries that extract information from a region of genome, for example a gene. Becuase gnomAD BigQuery tables utilize [integer range partitioning](https://cloud.google.com/bigquery/docs/creating-integer-range-partitions) they are optimized for this type of queries.

The main requirement to use this feature is to limit queries to a particular region by adding these conditions to the `WHERE` clause:

`WHERE start_position >= X AND start_position <= Y`

Where `[X, Y]` the region of interst. 

You can find values of `X` and `Y` by refering to an external databses. For example following table sumarizes the start and end positions for 4 genes on chromosome 17 extracted from an external resource:

| Gene 	| X 	| Y 	| Source 	|
|:-:	|-	|-	|-	|
| BRCA1 	| 43044295 	| 43125364 	| [link](https://ghr.nlm.nih.gov/gene/BRCA1#location) 	|
| COL1A1 	| 50184096 	| 50201649 	| [link](https://ghr.nlm.nih.gov/gene/COL1A1#location) 	|
| TP53 	| 31094927 	| 31377677 	| [link](https://ghr.nlm.nih.gov/gene/TP53#location) 	|
| NF1 	| 56593699 	| 56595611 	| [link](https://ghr.nlm.nih.gov/gene/NF1#location) 	|

Alternatively you could use the following query that extract the same infomration directly from gnomAD tables. 

In the following example we are using `BRCA1` on `chr17` as example, you could enter your gene of interest to modify all the following queries. Make sure for your gene of interest you are querying the right table (chromosome). If your query returns `NaN` this might be becuase you are querying the wrong table.

Also you could choose which version of gnomAD dataset you'd like to user for all the following queries:
 * `v2_1_1_exomes`
 * `v2_1_1_genomes`
 * `v3_genomes`


In [4]:
import ipywidgets as widgets

print("Variables for Region (Type 1) Queries")

gnomad_version_widget_region = widgets.Dropdown(
    options=['v2_1_1_exomes', 'v2_1_1_genomes', 'v3_genomes'],
    value='v3_genomes',
    description='gnomAD version:',
    disabled=False,
    style={'description_width': 'initial'}
)

display(gnomad_version_widget_region)

chromosome_widget_region = widgets.Dropdown(
    options=['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
             'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
             'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22',
             'chrX', 'chrY'],
    value='chr17',
    description='Chromosome:',
    disabled=False,
    style={'description_width': 'initial'}
)

display(chromosome_widget_region)

gene_symbol_widget_region= widgets.Text(
    value='BRCA1',
    placeholder='gene_symbol',
    description='Gene Symbol:',
    disabled=False,
    style={'description_width': 'initial'}
)

display(gene_symbol_widget_region)


Variables for Region (Type 1) Queries


Dropdown(description='gnomAD version:', index=2, options=('v2_1_1_exomes', 'v2_1_1_genomes', 'v3_genomes'), st…

Dropdown(description='Chromosome:', index=16, options=('chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7',…

Text(value='BRCA1', description='Gene Symbol:', placeholder='gene_symbol', style=DescriptionStyle(description_…

In [5]:
# Set the variables for the rest of the Type 1 queries based on the values above.
gnomad_version_region=gnomad_version_widget_region.value
chromosome_region=chromosome_widget_region.value
gene_symbol_region=gene_symbol_widget_region.value
print('Running Region (Type 1) queries on gnomAD version: {}, chromosome: {}, gene symbol: {}'.format(
    gnomad_version_region,
    chromosome_region,
    gene_symbol_region
))


Running Region (Type 1) queries on gnomAD version: v3_genomes, chromosome: chr17, gene symbol: BRCA1


In [6]:
from google.cloud import bigquery

client = bigquery.Client()

def run_query(query):
    query_job = client.query(query)
    result = query_job.to_dataframe(progress_bar_type='tqdm_notebook')
    gb_processed = (query_job.total_bytes_billed / 1024 ** 3)
    print('This query processed {} GB of data'.format(gb_processed))
    return result

In [7]:
query_template = """
SELECT MIN(start_position) AS X, MAX(end_position) AS Y
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table
WHERE EXISTS
  (SELECT 1 FROM UNNEST(main_table.alternate_bases) AS alternate_bases
   WHERE EXISTS (SELECT 1 from alternate_bases.vep WHERE SYMBOL = '{GENE}'))
"""
query = query_template.format(GNOMAD_VER=gnomad_version_region,
                              CHROM=chromosome_region,
                              GENE=gene_symbol_region)

#limits = client.query(query).to_dataframe()
limits = run_query(query)

print(limits)
x = limits.at[0, 'X']
y = limits.at[0, 'Y']


This query processed 1.0771484375 GB of data
          X         Y
0  43039296  43175241


After you found the `[X, Y]` range for your gene of interst you can run *Type1* queries efficiently. Here are a couple of examples:

### Query 1.1a - Variant Type (BigQuery)
Find the number of indels and snvs in the region of interest using BigQuery

In [8]:
# NOTE: For v2_1_1 the "variant_type" column must be replaced with "alternate_bases.allele_type AS variant_type"
query_template = """
SELECT COUNT(1) AS num, variant_type
FROM (
SELECT DISTINCT 
       start_position,
       reference_bases,
       alternate_bases.alt,
       variant_type,
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table,
     main_table.alternate_bases AS alternate_bases
WHERE start_position >= {X} AND start_position <= {Y}
)
GROUP BY 2
ORDER BY 1 DESC
"""
query = query_template.format(GNOMAD_VER=gnomad_version_region,
                              CHROM=chromosome_region, X=x, Y=y)

summary = run_query(query)
summary.head()


This query processed 0.009765625 GB of data


,num,variant_type
0,28016,snv
1,6238,indel


### Query 1.1b - Variant Type (Python)
You can also find the number of indels and snvs in the region of interest by doing the aggregation and count in Python using the dataframe.

In [9]:
# NOTE: For v2_1_1 the "variant_type" column must be replaced with "alternate_bases.allele_type AS variant_type"
query_template = """
SELECT DISTINCT 
       start_position,
       reference_bases,
       alternate_bases.alt,
       variant_type,
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table,
     main_table.alternate_bases AS alternate_bases
WHERE start_position >= {X} AND start_position <= {Y}
ORDER BY 1,2
"""
query = query_template.format(GNOMAD_VER=gnomad_version_region,
                              CHROM=chromosome_region, X=x, Y=y)

summary_dataframe = run_query(query)

# Count the number of each variant type in Python instead of in BigQuery
print('Number of variants by type:')
for v in summary_dataframe.variant_type.unique():
  print('{}: {}'.format(v,
                        np.count_nonzero(summary_dataframe['variant_type'] == v)))


This query processed 0.009765625 GB of data
Number of variants by type:
snv: 28016
indel: 6238


Instead of aggregating the results in BigQuery to count the number of each variant type, we could return all rows and process them here. The following query adds a few more columns to the previous query. 

### Query 1.2 - Allele Count by Sex
A query to retrieve all variants in the region of interest along with `AN` and `AC` values split by sex.

 * `AN`: Total number of alleles in samples
 * `AC`: Alternate allele count for samples

In [10]:
# NOTE: For v2_1_1 the "variant_type" column must be replaced with "alternate_bases.allele_type AS variant_type"
query_template = """
SELECT DISTINCT 
       start_position,
       reference_bases,
       AN,
       AN_male,
       AN_female,
       alternate_bases.alt AS alt,
       variant_type,
       alternate_bases.AC AS AC,
       alternate_bases.AC_male AS AC_male,
       alternate_bases.AC_female AS AC_female,
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table,
     main_table.alternate_bases AS alternate_bases
WHERE start_position >= {X} AND start_position <= {Y}
ORDER BY 1,2
"""
query = query_template.format(GNOMAD_VER=gnomad_version_region,
                              CHROM=chromosome_region, X=x, Y=y)

stats_sex = run_query(query)
stats_sex.head()



This query processed 0.009765625 GB of data


,start_position,reference_bases,AN,AN_male,AN_female,alt,variant_type,AC,AC_male,AC_female
0,43039296,A,143340,69472,73868,T,snv,2,2,0
1,43039301,T,143316,69456,73860,C,snv,7,4,3
2,43039307,T,143302,69434,73868,C,snv,1,1,0
3,43039316,A,143326,69468,73858,G,snv,1,0,1
4,43039336,TAG,143244,69424,73820,T,indel,1,1,0


We can then perform further analysis on the dataframe such as filtering out variants with a low allele count (AC).

In [11]:
stats_sex_filtered_ac=stats_sex.loc[stats_sex['AC'] > 10]
stats_sex_filtered_ac.head()

,start_position,reference_bases,AN,AN_male,AN_female,alt,variant_type,AC,AC_male,AC_female
23,43039470,G,143292,69446,73846,A,snv,194,97,97
39,43039598,TG,143292,69442,73850,T,indel,144,68,76
54,43039687,T,143308,69474,73834,G,snv,46,17,29
59,43039719,A,143270,69436,73834,G,snv,21,12,9
61,43039742,C,143320,69466,73854,A,snv,14,5,9


Or we could filter to find variants that were most common in females that were not found in any male samples.

In [12]:
stats_sex_no_male=stats_sex.loc[stats_sex['AC_male'] == 0].sort_values(by=('AC_female'),
                                                                       ascending = False)
stats_sex_no_male.head(10)

,start_position,reference_bases,AN,AN_male,AN_female,alt,variant_type,AC,AC_male,AC_female
18860,43114391,T,139448,67386,72062,TC,indel,17,0,17
11212,43088452,G,142962,69200,73762,A,snv,16,0,16
33872,43173828,C,128298,61064,67234,CCCCCCA,indel,13,0,13
11343,43089177,G,143204,69408,73796,A,snv,11,0,11
5978,43066019,C,143274,69430,73844,T,snv,9,0,9
9579,43080462,C,143124,69360,73764,T,snv,9,0,9
14642,43100663,A,18456,7494,10962,ATGT,indel,7,0,7
30813,43160144,C,142904,69214,73690,T,snv,7,0,7
20085,43119415,G,143278,69440,73838,A,snv,7,0,7
24640,43135141,C,143364,69482,73882,T,snv,7,0,7


Instead of splitting `AN` and `AC` values by sex we can analyze ancestry.

### Query 1.3 - Allele Count by Ancestry
A query to retrieve all variants in the region of interest along with `AN` and `AC` values for the following ancestries:
* `afr`: African-American/African ancestry
* `amr`: Latino ancestry
* `eas`: East Asian ancestry
* `nfe`: Non-Finnish European ancestry

In [13]:
# NOTE: For v2_1_1 the "variant_type" column must be replaced with "alternate_bases.allele_type AS variant_type"
query_template = """
SELECT DISTINCT 
       start_position,
       reference_bases,
       AN_afr,
       AN_amr,
       AN_eas,
       AN_nfe,
       alternate_bases.alt AS alt,
       variant_type,
       alternate_bases.AC_afr AS AC_afr,
       alternate_bases.AC_amr AS AC_amr,
       alternate_bases.AC_eas AS AC_eas,
       alternate_bases.AC_nfe AS AC_nfe,
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table,
     main_table.alternate_bases AS alternate_bases
WHERE start_position >= {X} AND start_position <= {Y}
ORDER BY 1,2
"""
query = query_template.format(GNOMAD_VER=gnomad_version_region,
                              CHROM=chromosome_region, X=x, Y=y)

stats_ancestry = run_query(query)
stats_ancestry.head()



This query processed 0.009765625 GB of data


,start_position,reference_bases,AN_afr,AN_amr,AN_eas,AN_nfe,alt,variant_type,AC_afr,AC_amr,AC_eas,AC_nfe
0,43039296,A,42066,13650,3132,64586,T,snv,1,0,0,0
1,43039301,T,42054,13660,3130,64578,C,snv,4,0,0,3
2,43039307,T,42064,13652,3134,64572,C,snv,0,0,0,1
3,43039316,A,42054,13660,3132,64576,G,snv,0,0,0,1
4,43039336,TAG,42028,13650,3134,64566,T,indel,0,0,0,0


An example here would be to report the most common variant for each ancestry that was not present in any of the others.

In [14]:
stats_ancestry_amr=stats_ancestry.loc[
                                      (stats_ancestry['AC_amr'] > 0) &
                                      (stats_ancestry['AC_afr'] == 0) &
                                      (stats_ancestry['AC_eas'] == 0) &
                                      (stats_ancestry['AC_nfe'] == 0)].sort_values(by=('AC_amr'),
                                                                                   ascending = False)
stats_ancestry_amr.head(10)


,start_position,reference_bases,AN_afr,AN_amr,AN_eas,AN_nfe,alt,variant_type,AC_afr,AC_amr,AC_eas,AC_nfe
19552,43117514,C,41940,13624,3130,64538,T,snv,0,116,0,0
25105,43136769,A,42036,13646,3130,64548,T,snv,0,115,0,0
2534,43051138,G,42044,13646,3134,64578,A,snv,0,109,0,0
27664,43146696,A,42050,13652,3132,64556,G,snv,0,64,0,0
22413,43127095,C,42054,13664,3126,64560,A,snv,0,58,0,0
17425,43109147,T,41976,13610,3132,64480,G,snv,0,55,0,0
29698,43155652,C,42026,13646,3128,64566,A,snv,0,36,0,0
21767,43124833,G,41994,13626,3130,64542,A,snv,0,35,0,0
8878,43077467,T,42000,13630,3128,64522,A,snv,0,32,0,0
33789,43173619,T,42024,13646,3134,64550,G,snv,0,32,0,0


### Query 1.4 - Burden of Mutation
Given a region of interest, compute the burden of mutation for the gene along with other summary statistics.

In [15]:
query_template = """
WITH summary_stats AS (
SELECT
  COUNT(1) AS num_variants,
  SUM(ARRAY_LENGTH(alternate_bases)) AS num_alts,  # This data appears to be bi-allelic.
  SUM((SELECT alt.AC FROM UNNEST(alternate_bases) AS alt)) AS sum_AC,
  APPROX_QUANTILES((SELECT alt.AC FROM UNNEST(alternate_bases) AS alt), 10) AS quantiles_AC,
  SUM(AN) AS sum_AN,
  APPROX_QUANTILES(AN, 10) AS quantiles_AN,
  -- Also include some information from Variant Effect Predictor (VEP).
  STRING_AGG(DISTINCT (SELECT annot.symbol FROM UNNEST(alternate_bases) AS alt,
                                                UNNEST(vep) AS annot LIMIT 1), ', ') AS genes
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table
WHERE start_position >= {X} AND start_position <= {Y})
---
--- The resulting quantiles and burden_of_mutation score give a very rough idea of the mutation
--- rate within these particular regions of the genome. This query could be further refined to
--- compute over smaller windows within the regions of interest and/or over different groupings
--- of AC and AN by population.
---
SELECT
  ROUND(({Y} - {X}) / num_variants, 3) AS burden_of_mutation,
  *,
FROM summary_stats
"""
query = query_template.format(GNOMAD_VER=gnomad_version_region,
                              CHROM=chromosome_region, X=x, Y=y)

stats = run_query(query)
stats.head()


This query processed 0.009765625 GB of data


,burden_of_mutation,num_variants,num_alts,sum_AC,quantiles_AC,sum_AN,quantiles_AN,genes
0,3.969,34254,34254,16394200,"[0, 0, 1, 1, 1, 1, 2, 3, 5, 17, 143046]",4625933720,"[274, 124060, 140064, 142458, 142938, 143116, ...","BRCA1, NBR2, NBR1, AC060780.1, AC060780.2, AC0..."


# gnomAD Queries Type2: Explore an entire chromosome

This section queries across an entire chromosome.


In [16]:
import ipywidgets as widgets

print("Variables for Chromosome (Type 2) queries")

gnomad_version_widget_chr = widgets.Dropdown(
    options=['v2_1_1_exomes', 'v2_1_1_genomes', 'v3_genomes'],
    value='v2_1_1_exomes',
    description='gnomAD version:',
    disabled=False,
    style={'description_width': 'initial'}
)

display(gnomad_version_widget_chr)

chromosome_widget_chr = widgets.Dropdown(
    options=['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
             'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
             'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22',
             'chrX', 'chrY'],
    value='chr17',
    description='Chromosome:',
    disabled=False,
    style={'description_width': 'initial'}
)

display(chromosome_widget_chr)



Variables for Chromosome (Type 2) queries


Dropdown(description='gnomAD version:', options=('v2_1_1_exomes', 'v2_1_1_genomes', 'v3_genomes'), style=Descr…

Dropdown(description='Chromosome:', index=16, options=('chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7',…

In [17]:
# Set the variables for the rest of the Chromosome (Type 2) queries based on the values above.
gnomad_version_chr=gnomad_version_widget_chr.value
chromosome_chr=chromosome_widget_chr.value
print('Running chromosome (Type 2) queries on gnomAD version: {}, chromosome: {}'.format(
    gnomad_version_chr,
    chromosome_chr
))

Running chromosome (Type 2) queries on gnomAD version: v2_1_1_exomes, chromosome: chr17


## Query 2.1 - Top SNV by Sex
Find 10,000 SNV on selected chromosome that are more common in women than men, min sample size set to 30.

In [18]:
query_template = """
SELECT DISTINCT 
       start_position AS str_pos,
       reference_bases AS ref,
       alternate_bases.alt AS alt,
       alternate_bases.allele_type AS type,
       vep.SYMBOL AS gene,
       vep.feature_type AS f_type,
       alternate_bases.AC AS AC,
       alternate_bases.AC_female AS AC_f,
       alternate_bases.AC_male AS AC_m,
       ROUND(alternate_bases.AC_female / alternate_bases.AC, 3) AS f_ratio
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE alternate_bases.AC > 30 AND vep.SYMBOL IS NOT NULL
ORDER BY f_ratio DESC
LIMIT 10000
"""

query = query_template.format(GNOMAD_VER=gnomad_version_chr,
                              CHROM=chromosome_chr)

stats_chr_sex = run_query(query)
stats_chr_sex.head()


This query processed 0.240234375 GB of data


,str_pos,ref,alt,type,gene,f_type,AC,AC_f,AC_m,f_ratio
0,79140505,A,C,snv,AATK-AS1,Transcript,41,39,2,0.951
1,79140505,A,C,snv,AATK,Transcript,41,39,2,0.951
2,33998749,T,TC,ins,AP2B1,Transcript,37,35,2,0.946
3,7221462,T,G,snv,RP11-542C16.2,Transcript,80,75,5,0.938
4,7221462,T,G,snv,GPS2,Transcript,80,75,5,0.938


We can condensed the result and only list gene symbols and the number of variants found in the query1.

In [19]:
stats_chr_sex.groupby('gene').count()[['str_pos']].sort_values(by=['str_pos'],
                                                               ascending=False).head()

,str_pos
gene,
DNAH17,72
CTC-297N7.11,65
RP11-799N11.1,61
RNF213,58
DNAH9,55


## Query 2.2 - Top SNV by ancenstry difference
Find top 1,000 SNV on selected chromosome that show the most significant differences between male samples of African-American ancestry versus Finnish ancestry

In [20]:
query_template = """
SELECT DISTINCT 
       start_position AS str_pos,
       reference_bases AS ref,
       alternate_bases.alt AS alt,
       alternate_bases.allele_type AS type,
       vep.SYMBOL AS gene,
       vep.feature_type AS f_type,
       alternate_bases.AC_male AS AC_m,
       alternate_bases.AC_fin_male AS AC_fin_m,
       alternate_bases.AC_afr_male AS AC_afr_m,
       ROUND(ABS(alternate_bases.AC_fin_male - alternate_bases.AC_afr_male) / alternate_bases.AC_male, 3) AS fin_afr_diff
FROM `bigquery-public-data.gnomAD.{GNOMAD_VER}__{CHROM}` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE vep.SYMBOL IS NOT NULL AND
      alternate_bases.AC_male > 20 AND alternate_bases.AC_fin_male > 0 AND alternate_bases.AC_afr_male > 0
order by fin_afr_diff DESC
LIMIT 1000
"""

query = query_template.format(GNOMAD_VER=gnomad_version_chr,
                              CHROM=chromosome_chr)

stats_chr_ancestry = run_query(query)
stats_chr_ancestry.head()


This query processed 0.240234375 GB of data


,str_pos,ref,alt,type,gene,f_type,AC_m,AC_fin_m,AC_afr_m,fin_afr_diff
0,76557038,G,A,snv,DNAH17,Transcript,56,53,1,0.929
1,76116856,C,T,snv,TMC6,Transcript,247,228,1,0.919
2,6900268,G,A,snv,RP11-589P10.5,Transcript,34,32,1,0.912
3,6900268,G,A,snv,ALOX12,Transcript,34,32,1,0.912
4,6900268,G,A,snv,RP11-589P10.7,Transcript,34,32,1,0.912


## Query 2.3 - Highest number of SNV
Find top 1000 genes with the highest number of SNV on selected chromosome.

In [21]:
query_template = """
SELECT gene, count(1) AS num_snv
FROM
(
SELECT DISTINCT 
       start_position AS str_pos,
       alternate_bases.alt AS alt,
       vep.SYMBOL AS gene,
FROM `bigquery-public-data.gnomAD.v2_1_1_exomes__chr17` AS main_table,
     main_table.alternate_bases AS alternate_bases,
     alternate_bases.vep AS vep
WHERE vep.SYMBOL IS NOT NULL AND alternate_bases.allele_type = 'snv'
)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1000
"""

query = query_template.format(GNOMAD_VER=gnomad_version_chr,
                              CHROM=chromosome_chr)

stats_chr_snv = run_query(query)
stats_chr_snv.head()


This query processed 0.091796875 GB of data


,gene,num_snv
0,CTC-297N7.11,9589
1,DNAH17,9208
2,RP11-799N11.1,9190
3,RNF213,6561
4,DNAH2,6361
